# Задача 

В следующей ячейке написал код по обучению модели машинного обучения для классификации Ирисов.
Необходимо реализовать веб-сервис на Flask и обработчик на Celery таким образом, чтобы получившаяся система позволяла использовать эту модель для классификации через сеть. 

In [58]:
from sklearn.datasets import load_iris
import pickle
from sklearn.linear_model import LogisticRegression


X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)

In [59]:
with open('clf.pickle', 'wb') as f:
    f.write(pickle.dumps(clf))

In [102]:
%%writefile server.py
import pickle
import json
import numpy as np
from flask import Flask, request
from celery import Celery
from celery.result import AsyncResult

celery_app = Celery('server', backend='redis://localhost', broker='redis://localhost')  # и брокер и база - redis
app = Flask(__name__)


def load_model(pickle_path):
    with open(pickle_path, 'rb') as f:
        raw_data = f.read()
        model = pickle.loads(raw_data)
    return model

model = load_model('clf.pickle')

@celery_app.task
def classify_iris(iris_data):
    result = model.predict(iris_data)
    return int(result[0])


@app.route('/iris', methods=["GET", "POST"])
def iris_handler():
    if request.method == 'POST':
        data = request.get_json(force=True)
        iris_data = np.reshape(data['iris'], (1, -1)).tolist()  # needs to be serializable
        task = classify_iris.delay(iris_data)
        response = {
            "task_id": task.id
        }
        
        return json.dumps(response)

@app.route('/iris/<task_id>')
def iris_check_handler(task_id):
    task = AsyncResult(task_id, app=celery_app)
    if task.ready():
        response = {
            "status": "DONE",
            "result": task.result
        }
    else:
        response = {
            "status": "IN_PROGRESS"
        }
    return json.dumps(response)
    
if __name__ == '__main__':
    app.run("0.0.0.0", 8000)  # Запускаем сервер на 8000 порту

Overwriting server.py


In [100]:
! start-worker.sh

Success!


In [101]:
! launch-server.sh server.py

Success!


После того, как вы реализуете свой веб-сервис, достаточно будет его запустить и нажать кнопку "Отправить решение". После нажатия автоматически запустится скрипт `check-server.py`, который создаст файл `result.json`. 

Сам скрипт можно использовать для проверки корректности своего решения.

In [2]:
! cat $(which check-server.py)

#!/usr/bin/env python3

import requests
import json
import time

questions = [
    [4.6, 3.1, 1.5, 0.2],
    [5.2, 2.7, 3.9, 1.4],
    [6.9, 3.1, 5.1, 2.3]
]

result = []
for q in questions:
    data = {
        'iris': q
    }

    response = requests.post("http://localhost:8000/iris", json=data)
    task_id = response.json()['task_id']
    status = "IN_PROGRESS"
    while status != "DONE":
        time.sleep(2.0)
        r = requests.get('http://localhost:8000/iris/{}'.format(task_id))
        status = r.json()['status']

    result.append(r.json()['result'])

with open('/home/jovyan/work/result.json', 'w') as f:
    f.write(json.dumps(result, indent=4))